In [3]:
import ast
import pandas as pd
import numpy as np
from tqdm import tqdm
import ast
wdsq = pd.read_csv('wdsq_test_dates.csv')

In [36]:
wdsq.loc[25, 'ids_one_hops_dict_en_bela'] = str({'Q605122': 'P131, P31, P1417, P590, P2053, P974, P366, P3858, P1225, P373, P2043, P8189, P625, P403, P910, P885, P18, P15, P646, P17, P1343, P4614, P244'})

In [13]:
for k in tqdm(range(len(wdsq))):
    
    inds = np.argsort(ast.literal_eval(wdsq.loc[k, 'bela_base'])[0]['el_scores'])[::-1]

    ents_sort =[ast.literal_eval(wdsq.loc[k, 'bela_base'])[0]['entities'][i] for i in inds]
    offsets_sort = [ast.literal_eval(wdsq.loc[k, 'bela_base'])[0]['offsets'][i] for i in inds]
    lengths_sort = [ast.literal_eval(wdsq.loc[k, 'bela_base'])[0]['lengths'][i] for i in inds]
    el_scores = [ast.literal_eval(wdsq.loc[k, 'bela_base'])[0]['el_scores'][i] for i in inds]

    cur_set = set()

    fin_ents = []
    fin_offsets = []
    fin_lengths = []
    fin_el_scores = []

    for j in range(len(ents_sort)):
        if ents_sort[j] not in cur_set:
            cur_set.add(ents_sort[j])
            fin_ents.append(ents_sort[j])
            fin_offsets.append(offsets_sort[j])
            fin_lengths.append(lengths_sort[j])
            fin_el_scores.append(el_scores[j])
            
    start = fin_offsets[0]
    length = fin_lengths[0]
    
    wdsq.loc[k, 'question_en_rank'] = wdsq.loc[k, 'question'][:start]+'<e>'+wdsq.loc[k, 'question'][start+length:]
    wdsq.loc[k, 'scores_en_rank'] = ", ".join(map(str, fin_el_scores))


100%|██████████| 824/824 [00:07<00:00, 105.45it/s]


In [15]:
wdsq.columns

Index(['question', 'sparql', 'entity', 'relation', 'answer', 'answer_lbl',
       'bela_base', 'bela_base_ents', 'question_en_rank', 'scores_en_rank',
       'ids_one_hops_dict_en'],
      dtype='object')

In [4]:
questions = []
paths = []
scores = []
origin_questions = []
relations = []

for k in range(len(wdsq)):
    questions.append(wdsq.loc[k, 'question_en_rank'])
    cur_dict = ast.literal_eval(wdsq.loc[k, 'ids_one_hops_dict_en'])
    for key in cur_dict.keys():
        cur_dict[key] = list(cur_dict[key].split(', '))
    paths.append(cur_dict)
    leng = len(list(wdsq.loc[k, 'scores_en_rank'].split(', ')))
    scores.append(list(wdsq.loc[k, 'scores_en_rank'].split(', ')))
    origin_questions.append(wdsq.loc[k, 'question'])
for q in range(len(wdsq)):
    rels_set = set()
    for key in paths[q]:
        for elem in paths[q][key]:
            rels_set.add(elem)
    relations.append(list(rels_set))

In [12]:
all_re = pd.read_csv('wikidata_properties.csv')
all_re = all_re[:11233]

In [13]:
rel_dict = dict()
for i in range(len(all_re)):
    cur_id = all_re.loc[i, 'ID']
    cur_lbl = all_re.loc[i, 'label']
    rel_dict[cur_id] = cur_lbl

In [7]:
import time
import requests

In [36]:
paths[0].keys()

dict_keys(['Q3272', 'Q23397', 'Q4022', 'Q2135'])

In [8]:
cur_rel = list(paths[0].keys())[0]
r = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids={}&languages=en&formatversion=2&format=json".format(cur_rel))
data = r.json()
data['entities'][cur_rel]['labels']['en']['value']

'Kaley Cuoco'

In [9]:
for p in tqdm(range(len(paths))):
    for key in paths[p].keys():
        for w in range(len(paths[p][key])):
            if paths[p][key][w] in rel_dict:
                paths[p][key][w] = rel_dict[paths[p][key][w]]
            else:
                time.sleep(1)
                if paths[p][key][w] != '':
                    cur_rel = paths[p][key][w]
                    r = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids={}&languages=en&formatversion=2&format=json".format(cur_rel))
                    data = r.json()
                    value = data['entities'][cur_rel]['labels']['en']['value']
                    paths[p][key][w] = value

100%|██████████| 57/57 [01:04<00:00,  1.12s/it]


In [7]:
paths[0]

{'Q3272': ['area',
  'Online PWN Encyclopedia ID',
  'perimeter',
  'NL CR AUT ID',
  'described by source',
  'vertical depth',
  'located in the administrative territorial entity',
  'lake outflow',
  'has part(s)',
  'VIAF ID',
  'volume as quantity',
  'Great Russian Encyclopedia Online ID (old version)',
  'Interlingual Index ID',
  'watershed area',
  'Freebase ID',
  'Getty Thesaurus of Geographic Names ID',
  'Encyclopedia of China (Third Edition) ID',
  'KBpedia ID',
  'locator map image',
  'Store norske leksikon ID',
  'different from',
  'OpenStreetMap relation ID',
  'Commons category',
  'coordinate location',
  'country',
  'Gran Enciclopèdia Catalana ID',
  'Canadian Encyclopedia article ID',
  'GeoNames ID',
  'drainage basin',
  'inflows',
  'on focus list of Wikimedia project',
  'CGNDB unique ID',
  'category for maps',
  'elevation above sea level',
  'Encyclopædia Britannica Online ID',
  'residence time of water',
  'length',
  'follows',
  'image',
  'instance o

In [10]:
for p in tqdm(range(len(relations))):
    for q in range(len(relations[p])):
        if relations[p][q] in rel_dict:
            relations[p][q] = rel_dict[relations[p][q]]
        else:
            time.sleep(1)
            if relations[p][q] != '':
                cur_rel = relations[p][q]
                r = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids={}&languages=en&formatversion=2&format=json".format(cur_rel))
                data = r.json()
                value = data['entities'][cur_rel]['labels']['en']['value']
                relations[p][q] = value

100%|██████████| 57/57 [01:00<00:00,  1.07s/it]


In [9]:
print(1)

1


In [11]:
import json
test_json=dict()
test_json["questions"]=questions
test_json["paths"]=paths
test_json["scores"]=scores
test_json["origin_questions"]=origin_questions
test_json["relations"]=relations

with open('test_mintaka_dates.json', 'w') as f:
    json.dump(test_json, f,ensure_ascii=False)

In [12]:
import json

with open('test_mintaka_dates.json') as user_file:
    parsed_json = json.load(user_file)

In [13]:
from tqdm import tqdm
paths = parsed_json['paths']

In [14]:
del_list = []
for p in tqdm(range(len(parsed_json['paths']))):
    for key in parsed_json['paths'][p].keys():
        for w in range(len(parsed_json['paths'][p][key])):
            if parsed_json['paths'][p][key][w] == '':
                del_list.append([p, key])

100%|██████████| 57/57 [00:00<00:00, 13558.40it/s]


In [15]:
for i in range(len(del_list)):
    id1 = del_list[i][0]
    id2 = del_list[i][1]
    del parsed_json['paths'][id1][id2]

In [16]:
with open('test_wdsq.json', 'w') as f:
    json.dump(parsed_json, f,ensure_ascii=False)

In [20]:
! python3 PLMs-in-Practical-KBQA/main/rel_similarity/predict.py

loaded tokenizer!
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_tr

loaded model!
��Ԥ�⣺ 500 / 824
Ԥ����ȣ� 0.6067961165048543


### Mintaka

In [ ]:
import pandas as pd
mintaka = pd.read_csv('mintaka_paper_res_sel_rank.csv')

In [74]:
questions = []
paths = []
scores = []
origin_questions = []
relations = []

for k in range(len(mintaka)):
    questions.append(mintaka.loc[k, 'question_en_rank'])
    cur_dict = ast.literal_eval(mintaka.loc[k, 'ids_one_hops_dict_en'])
    for key in cur_dict.keys():
        cur_dict[key] = list(cur_dict[key].split(', '))
    paths.append(cur_dict)
    scores.append(list(mintaka.loc[k, 'scores_en_rank'].split(', ')))
    origin_questions.append(mintaka.loc[k, 'question'])
for q in range(len(mintaka)):
    rels_set = set()
    for key in paths[q]:
        for elem in paths[q][key]:
            rels_set.add(elem)
    relations.append(list(rels_set))

In [75]:
import requests
import time

for p in tqdm(range(len(paths))):
    for key in paths[p].keys():
        for w in range(len(paths[p][key])):
            if paths[p][key][w] in rel_dict:
                paths[p][key][w] = rel_dict[paths[p][key][w]]
            else:
                time.sleep(1)
                if paths[p][key][w] != '':
                    cur_rel = paths[p][key][w]
                    r = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids={}&languages=en&formatversion=2&format=json".format(cur_rel))
                    data = r.json()
                    value = data['entities'][cur_rel]['labels']['en']['value']
                    paths[p][key][w] = value

100%|██████████| 340/340 [00:45<00:00,  7.45it/s]


In [76]:
del_list = []
for p in tqdm(range(len(parsed_json['paths']))):
    for key in parsed_json['paths'][p].keys():
        for w in range(len(parsed_json['paths'][p][key])):
            if parsed_json['paths'][p][key][w] == '':
                del_list.append([p, key])

100%|██████████| 1186/1186 [00:00<00:00, 10298.76it/s]


In [78]:
for p in tqdm(range(len(relations))):
    for q in range(len(relations[p])):
        if relations[p][q] in rel_dict:
            relations[p][q] = rel_dict[relations[p][q]]
        else:
            time.sleep(1)
            if relations[p][q] != '':
                cur_rel = relations[p][q]
                r = requests.get("https://www.wikidata.org/w/api.php?action=wbgetentities&props=labels&ids={}&languages=en&formatversion=2&format=json".format(cur_rel))
                data = r.json()
                value = data['entities'][cur_rel]['labels']['en']['value']
                relations[p][q] = value

100%|██████████| 340/340 [00:45<00:00,  7.47it/s]


In [79]:
import json
test_json=dict()
test_json["questions"]=questions
test_json["paths"]=paths
test_json["scores"]=scores
test_json["origin_questions"]=origin_questions
test_json["relations"]=relations

with open('test_mintaka.json', 'w') as f:
    json.dump(test_json, f,ensure_ascii=False)

In [1]:
! python3 PLMs-in-Practical-KBQA/main/rel_similarity/predict.py

loaded tokenizer!
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_lay

loaded model!


In [80]:
rubq = pd.read_csv('rubq_paper_res_sel_rank.csv')

In [79]:
rubq.to_csv('rubq_paper_res_sel_rank.csv', index = False)

In [35]:
len(rubq)

1186

In [81]:
with open('model/distilbert-base-uncased/rubq_en/result.json') as user_file:
    res = json.load(user_file)

In [37]:
len(res['re_scores'])

1186

In [82]:
for i in tqdm(range(len(res['re_scores']))):
    subj, rel = best_label(res, i)
    rubq.loc[i, 'subj_rank'] = subj
    rubq.loc[i, 'rel_rank'] = rel

100%|██████████| 1186/1186 [00:00<00:00, 4468.00it/s]


In [83]:
for i in tqdm(range(len(rubq))):
    lbl = rubq.loc[i, 'rel_rank']
    rubq.loc[i, 'rel_rank_id'] = rel_dict_rev[lbl]

100%|██████████| 1186/1186 [00:00<00:00, 11516.73it/s]


In [84]:
count = 0
for i in range(len(rubq)):
    true_id = rubq.loc[i, 'props_labs']
    pred_id = rubq.loc[i, 'rel_rank_id']
    if true_id == pred_id:
        count += 1

In [85]:
count/len(rubq)

0.5657672849915683

In [46]:
count/len(rubq)

0.5733558178752108

In [47]:
rubq.to_csv('rubq_paper_res_sel_rank.csv', index = False)

In [148]:
rubq.columns

Index(['uid', 'question', 'subject', 'props', 'query', 'tag', 'answers',
       'pred_relation', 'pred_obj', 'pred_on_wdsq_and_rubq_for_rubq_ner_upd',
       'ans_type', 'ans_correct', 'object_gold', 'pred_prop_fast',
       'pred_joined', 'max_pred_label', 'props_labs', 'max_pred_id',
       'pred_init', 'pred_fast', 'pred_t5', 'pred_combo', 'pred_joined_sel',
       'pred_ids', 'appr', 'bela_base_ents', 'pred_bela_ner_ru',
       'mgenre_with_bela_en_ques', 'mgenre_with_bela_ru_ques', 'bela_base',
       'question_text', 'bela_base_ru', 'scores_bela_ner_ru',
       'pred_prop_fast_en', 'pred_prop_fast_ru', 'pred_obj_fall_init_ru',
       'pred_obj_fall_init_en', 'pred_fin_en', 'pred_fin_ru', 'ents_falcon',
       'rels_falcon', 'question_en_rank', 'scores_en_rank',
       'ids_one_hops_dict_en', 'subj_rank', 'rel_rank', 'rel_rank_id'],
      dtype='object')

In [56]:
def run_query_object(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

   wd:<ENTITY> wdt:<PROPERTY> ?object .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['object']['value'])                 
    return ', '.join(objects)

In [86]:
for i in tqdm(range(len(rubq))):
    subj = rubq.loc[i, 'subj_rank']
    rel = rubq.loc[i, 'rel_rank_id']
    try:
        time.sleep(1)
        ans = run_query_object(subj, rel)
        rubq.loc[i, 'ans_rank'] = ans
    except:
        continue

100%|██████████| 1186/1186 [25:35<00:00,  1.30s/it]


In [87]:
for j in tqdm(range(len(rubq))):
    cands = rubq.loc[j, 'ans_rank'].split(', ')
    elems = []
    for k in range(len(cands)):
        if 'entity/' in cands[k]:
            elems.append(cands[k].split('entity/')[1])
        else:
            elems.append(cands[k])
    rubq.loc[j, 'ans_rank_clean'] = ', '.join(elems)

100%|██████████| 1186/1186 [00:00<00:00, 9601.01it/s]


In [91]:
count = 0
for i in range(len(rubq)):
    if rubq.loc[i, 'answers'] == rubq.loc[i, 'ans_rank_clean']:
        count += 1
print(count/len(rubq))

0.4527824620573356


In [76]:
count = 0
for i in range(len(rubq)):
    if rubq.loc[i, 'answers'] == rubq.loc[i, 'pred_obj_maxpred_lbl']:
        count += 1
print(count/len(rubq))

0.4620573355817875


In [74]:
rubq.columns

Index(['Unnamed: 0', 'uid', 'question', 'subject', 'props', 'query', 'tag',
       'answers', 'pred_relation', 'pred_obj',
       'pred_on_wdsq_and_rubq_for_rubq_ner_upd', 'ans_type', 'ans_correct',
       'object_gold', 'pred_prop_fast', 'pred_joined', 'max_pred_label',
       'props_labs', 'max_pred_id', 'pred_init', 'pred_fast', 'pred_t5',
       'pred_combo', 'pred_joined_sel', 'pred_ids', 'appr', 'bela_base_ents',
       'pred_bela_ner_ru', 'mgenre_with_bela_en_ques',
       'mgenre_with_bela_ru_ques', 'bela_base', 'question_text',
       'bela_base_ru', 'scores_bela_ner_ru', 'pred_prop_fast_en',
       'pred_prop_fast_ru', 'pred_obj_fall_init_ru', 'pred_obj_fall_init_en',
       'pred_fin_en', 'pred_fin_ru', 'ents_falcon', 'rels_falcon',
       'question_en_rank', 'scores_en_rank', 'ids_one_hops_dict_en',
       'subj_rank', 'rel_rank', 'rel_rank_id', 'ans_rank', 'ans_rank_clean',
       'pred_obj_maxpred', 'ans_class_rank', 'pred_obj_maxpred_lbl',
       'pred_rel_rank'],
      

In [89]:
for i in range(len(rubq)):
    if pd.notna(rubq.loc[i, 'pred_obj_maxpred_lbl']):
        rubq.loc[i, 'pred_rel_rank'] = rubq.loc[i, 'pred_obj_maxpred_lbl']
    else:
        rubq.loc[i, 'pred_rel_rank'] = rubq.loc[i, 'ans_rank_clean']

In [90]:
count = 0
for i in range(len(rubq)):
    if rubq.loc[i, 'answers'] == rubq.loc[i, 'pred_rel_rank']:
        count += 1
print(count/len(rubq))

0.5101180438448567


In [ ]:
rubq.

In [54]:
count = 0
for i in range(len(rubq)):
    if rubq.loc[i, 'answers'] == rubq.loc[i, 'pred_obj_fall_init_en']:
        count += 1
print(count/len(rubq))

0.0


In [167]:
rubq.columns

Index(['uid', 'question', 'subject', 'props', 'query', 'tag', 'answers',
       'pred_relation', 'pred_obj', 'pred_on_wdsq_and_rubq_for_rubq_ner_upd',
       'ans_type', 'ans_correct', 'object_gold', 'pred_prop_fast',
       'pred_joined', 'max_pred_label', 'props_labs', 'max_pred_id',
       'pred_init', 'pred_fast', 'pred_t5', 'pred_combo', 'pred_joined_sel',
       'pred_ids', 'appr', 'bela_base_ents', 'pred_bela_ner_ru',
       'mgenre_with_bela_en_ques', 'mgenre_with_bela_ru_ques', 'bela_base',
       'question_text', 'bela_base_ru', 'scores_bela_ner_ru',
       'pred_prop_fast_en', 'pred_prop_fast_ru', 'pred_obj_fall_init_ru',
       'pred_obj_fall_init_en', 'pred_fin_en', 'pred_fin_ru', 'ents_falcon',
       'rels_falcon', 'question_en_rank', 'scores_en_rank',
       'ids_one_hops_dict_en', 'subj_rank', 'rel_rank', 'rel_rank_id',
       'ans_rank', 'ans_rank_clean'],
      dtype='object')

In [179]:
rubq.loc[14, 'ids_one_hops_dict_en']

"{'Q176772': 'P8419, P509, P463, P1559, P20, P1449, P22, P175, P569, P735, P170, P551, P460, P607, P157, P2563, P6262, P27, P7047, P734, P3417, P19, P1412, P25, P10757, P21, P8345, P1080, P7818, P11408, P18, P39, P1196, P570, P2561, P646, P69, P4584, P11196, P108, P941, P373, P3342, P31, P106, P1441, P5905', 'Q82799': 'P8834, P1687, P6573, P11567, P7827, P1282, P1709, P5008, P460, P5395, P9318, P1889, P2559, P279, P1225, P3417, P3827, P8189, P1557, P227, P2572, P7818, P8885, P18, P361, P2888, P527, P1296, P8168, P5198, P646, P1368, P1036, P3123, P4212, P910, P3219, P244, P1417, P9475, P373, P8313, P1014, P691, P1343, P2581, P8408, P4613, P8814', 'Q7565': 'P4946, P902, P950, P8309, P486, P1687, P7033, P9497, P268, P2892, P6417, P8015, P4527, P9937, P9318, P10913, P1889, P3827, P279, P3417, P8189, P5160, P227, P508, P443, P461, P18, P527, P1245, P2004, P1296, P8168, P6366, P8370, P8519, P5198, P646, P1552, P4500, P1036, P3984, P6293, P4212, P7870, P910, P244, P3916, P1417, P672, P10192, 

In [182]:
rubq[['max_pred_id']]

,max_pred_id
0,P828
1,P50
2,P50
3,P36
4,P1303
...,...
1181,P1299
1182,P50
1183,P170
1184,P138


In [57]:
def run_query_subject(entity_idx, property_idx):
    url = "https://query.wikidata.org/sparql"
    query = """

   SELECT ?object  WHERE {

   ?object wdt:<PROPERTY> wd:<ENTITY> .} 
        """.replace(
            "<ENTITY>", entity_idx).replace(
            "<PROPERTY>", property_idx)
     
    request = requests.get(url, params={"format": "json", "query": query})
    data = request.json()
    objects = []
    for q in range(len(data["results"]['bindings'])):
        objects.append(data["results"]['bindings'][q]['object']['value'])                 
    return ', '.join(objects)

In [62]:
rubq.head()

,Unnamed: 0,uid,question,subject,props,query,tag,answers,pred_relation,pred_obj,...,scores_en_rank,ids_one_hops_dict_en,subj_rank,rel_rank,rel_rank_id,ans_rank,ans_rank_clean,pred_obj_maxpred,ans_class_rank,pred_obj_maxpred_lbl
0,0,0,What can cause a tsunami?,Q8070,wdt:P828,SELECT ?answer \nWHERE {\n wd:Q8070 wdt:P828 ...,1-hop,"Q7944, Q60186, Q167903, Q2580904, Q5975740, Q7...",P2528,NaN,...,0.8407442569732666,"{'Q8070': 'P487, P948, P3221, P950, P486, P703...",Q8070,has cause,P828,"http://www.wikidata.org/entity/Q7944, http://w...","Q7944, Q60186, Q167903, Q2580904, Q5975740, Q7...","http://www.wikidata.org/entity/Q7944, http://w...","Q7944, Q60186, Q167903, Q2580904, Q5975740, Q7...","Q7944, Q60186, Q167903, Q2580904, Q5975740, Q7..."
1,1,1,"Who wrote the novel ""uncle Tom's Cabin""?",Q2222,wdt:P50,SELECT ?answer \nWHERE {\n wd:Q2222 wdt:P50 ?...,1-hop,Q102513,P50,Q102513,...,"0.38015487790107727, 0.06446564197540283, 0.01...","{'Q2222': 'P50, P747, P110, P950, P409, P136, ...",Q2222,author,P50,http://www.wikidata.org/entity/Q102513,Q102513,http://www.wikidata.org/entity/Q102513,Q102513,Q102513
2,2,2,"Who is the author of the play ""Romeo and Juliet""?",Q83186,wdt:P50,SELECT ?answer \nWHERE {\n wd:Q83186 wdt:P50 ...,1-hop,Q692,P50,Q692,...,"0.9108105301856995, 0.020021561533212662, 0.00...","{'Q83186': 'P2348, P50, P7439, P747, P8419, P8...",Q83186,author,P50,http://www.wikidata.org/entity/Q692,Q692,http://www.wikidata.org/entity/Q692,Q692,Q692
3,3,3,What is the name of the capital of Romania?,Q218,wdt:P36,SELECT ?answer \nWHERE {\n wd:Q218 wdt:P36 ?a...,1-hop,Q19660,P1376,NaN,...,"0.8396620154380798, 0.12966066598892212, 0.043...","{'Q218': 'P38, P902, P487, P948, P463, P8309, ...",Q218,capital,P36,http://www.wikidata.org/entity/Q19660,Q19660,http://www.wikidata.org/entity/Q19660,Q19660,Q19660
4,4,5,What instrument did Jimi Hendrix play?,Q5928,wdt:P1303,SELECT ?answer \nWHERE {\n wd:Q5928 wdt:P1303...,1-hop,"Q6607, Q483994, Q626035, Q2643890, Q17172850",P1303,"Q6607, Q483994, Q626035, Q2643890, Q17172850",...,"0.9030609130859375, 0.057044822722673416","{'Q5928': 'P11194, P463, P8041, P2722, P22, P4...",Q5928,instrument,P1303,"http://www.wikidata.org/entity/Q6607, http://w...","Q6607, Q483994, Q626035, Q17172850","http://www.wikidata.org/entity/Q6607, http://w...","Q6607, Q483994, Q626035, Q17172850","Q6607, Q483994, Q626035, Q17172850"


In [64]:
import numpy as np
rubq['pred_obj_maxpred'] = np.nan

In [65]:
for k in tqdm(range(len(rubq))):
    if pd.isna(rubq.loc[k, 'pred_obj_maxpred']):
        try:
            cur_subjects = rubq.loc[k, 'bela_base_ents'].split(', ')
            for w in range(len(cur_subjects)):
                time.sleep(1)
                cur_subj = rubq.loc[k, 'bela_base_ents'].split(', ')[w]
                cur_rel = rubq.loc[k, 'max_pred_id'].split(', ')[0]
                obj_pred = run_query_object(cur_subj, cur_rel)
                if len(obj_pred) > 0:
                    rubq.loc[k, 'pred_obj_maxpred'] = obj_pred
                    break
        except:
            time.sleep(1)
            continue

100%|██████████| 1186/1186 [49:37<00:00,  2.51s/it] 


In [66]:
for j in tqdm(range(len(rubq))):
    if pd.notna(rubq.loc[j, 'pred_obj_maxpred']):
        cands = rubq.loc[j, 'pred_obj_maxpred'].split(', ')
        elems = []
        for k in range(len(cands)):
            if 'entity/' in cands[k]:
                elems.append(cands[k].split('entity/')[1])
            else:
                elems.append(cands[k])
        rubq.loc[j, 'pred_obj_maxpred_lbl'] = ', '.join(elems)

100%|██████████| 1186/1186 [00:00<00:00, 13465.15it/s]


In [187]:
count = 0
for i in range(len(rubq)):
    ans = rubq.loc[i, 'ans_rank_clean']
    if ans == rubq.loc[i, 'answers']:
        count += 1
print(count/len(rubq))

0.4536256323777403


In [188]:
rubq.to_csv('rubq_paper_res_sel_rank.csv')